In [1]:
# 경로 지정 Code

In [1]:
# ✅ 셀 1: CARLA Python API 연결 및 기본 모듈 로드
import sys
import time
import carla
import random
from ultralytics import YOLO
import numpy as np
import cv2
# CARLA PythonAPI 경로 추가
sys.path.append(r"C:\Users\user\Carla-0.9.15\PythonAPI")
sys.path.append(r"C:\Users\user\Carla-0.9.15\PythonAPI\carla")
sys.path.append(r"C:\Users\user\Carla-0.9.15\PythonAPI\carla\agents\navigation")  # <- dao 경로

# 모듈 임포트
from agents.navigation.behavior_agent import BehaviorAgent
from agents.navigation.local_planner import RoadOption
from agents.navigation.global_route_planner import GlobalRoutePlanner
from global_route_planner_dao import GlobalRoutePlannerDAO
from carla import Transform, Location, Rotation

In [2]:
# ✅ 셀 2: CARLA 클라이언트 연결 및 맵 로드
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)

# 현재 맵 이름 확인
current_map = client.get_world().get_map().name
print(f"현재 로드된 맵: {current_map}")

# Town10HD_Opt가 아니라면 로드 시도
if "Town10HD" not in current_map:
    print("Town10HD_Opt 맵 로드 중...")
    world = client.load_world("Town10HD_Opt")
else:
    print("✅ Town10HD_Opt 맵이 이미 로드되어 있음.")
    world = client.get_world()

map = world.get_map()
blueprint_library = world.get_blueprint_library()

현재 로드된 맵: Carla/Maps/Town10HD
✅ Town10HD_Opt 맵이 이미 로드되어 있음.


In [4]:
spectator = world.get_spectator()
# vehicle_bp = blueprint_library.find("vehicle.tesla.model3")
# vehicle = world.try_spawn_actor(vehicle_bp, spectator.get_transform())
print(spectator.get_transform())
# 출발 좌표 Transform(Location(x=-25.583757, y=27.888950, z=2.027854), Rotation(pitch=1.108314, yaw=2.119219, roll=0.000001))
# 도착 좌표 Transform(Location(x=78.876656, y=30.494513, z=4.072402), Rotation(pitch=-6.831172, yaw=0.032746, roll=0.000011))

Transform(Location(x=-13.606251, y=16.739204, z=2.150975), Rotation(pitch=-1.106504, yaw=178.959442, roll=0.000086))


In [3]:
# ✅ 셀 3: 차량 스폰
spawn_points = map.get_spawn_points()
print(f"총 스폰 포인트 개수: {len(spawn_points)}")

# 차량 블루프린트 선택 및 스폰
bp = blueprint_library.find("vehicle.ford.ambulance")
start_transform = Transform(Location(x=-30.939667, y=27.588772, z=1.720956), Rotation(pitch=-9.529569, yaw=0.619764, roll=0.000031))
vehicle = world.try_spawn_actor(bp, start_transform)
if not vehicle:
    raise RuntimeError("🚨 차량 스폰 실패!")
print("✅ 차량 스폰 성공!")

# 카메라: 초기 위치 조정 (위에서 내려다보는 고정 시점)
spectator = world.get_spectator()
spectator.set_transform(carla.Transform(
    start_transform.location + carla.Location(z=50),
    carla.Rotation(pitch=-90)
))

AttributeError: type object 'map' has no attribute 'get_spawn_points'

In [14]:
# ✅ 셀 4: GlobalRoutePlanner 사용해 짧고 안전한 경로 생성

# 출발지와 도착지 Transform
start_transform = Transform(
    Location(x=-25.583757, y=27.888950, z=2.027854), 
    Rotation(pitch=1.108314, yaw=2.119219, roll=0.000001)
)
end_transform = Transform(
    Location(x=78.876656, y=30.494513, z=4.072402), 
    Rotation(pitch=-6.831172, yaw=0.032746, roll=0.000011))


# GlobalRoutePlanner 초기화 및 설정
dao = GlobalRoutePlannerDAO(map, sampling_resolution=2.0)
grp = GlobalRoutePlanner(dao, 2.0)

# 전체 경로 생성
route_all = grp.trace_route(start_transform.location, end_transform.location)
print(f"📍 생성된 경로 포인트 수: {len(route_all)}")


📍 생성된 경로 포인트 수: 55


In [15]:
# ✅ 셀 5: BehaviorAgent 생성 및 신호 무시 설정
agent = BehaviorAgent(vehicle, behavior='aggressive')  # 신호 무시하려면 aggressive 추천
agent.set_global_plan(route_all)

# 🚦 신호 무시 옵션 직접 수정 (안 되면 behavior 수정 필요)
agent._ignore_traffic_lights = True
agent._ignore_stop_signs = True
agent._ignore_lane_changes = True

print("✅ BehaviorAgent 설정 및 경로 적용 완료!")

✅ BehaviorAgent 설정 및 경로 적용 완료!


In [16]:
# ✅ 셀 6: 차량 주행 루프 + 조감도 카메라
print("🚗 차량 이동 시작!")

while True:
    if agent.done():
        print("✅ 목적지 도착 완료!")
        break

    control = agent.run_step()
    vehicle.apply_control(control)

    # 위에서 부드럽게 따라가는 카메라 (z 고정, yaw 고정)
    transform = vehicle.get_transform()
    vehicle_loc = transform.location
    spectator.set_transform(carla.Transform(
        carla.Location(x=vehicle_loc.x, y=vehicle_loc.y, z=50),
        carla.Rotation(pitch=-90, yaw=0)
    ))

    time.sleep(0.05)

🚗 차량 이동 시작!
✅ 목적지 도착 완료!


In [17]:
import random
import carla
import math
# 특정 위치 주변에 일반 차량을 겹치지 않게 스폰하고 자율주행을 활성화하는 함수.
def spawn_general_vehicles(world, blueprint_library, base_point, num_vehicles=10, min_distance=5.0):

    spawned_vehicles = []
    base_loc = base_point.location

    def is_too_close(existing_locations, new_location, min_dist):
        for loc in existing_locations:
            if new_location.distance(loc) < min_dist:
                return True
        return False

    attempted_locations = []

    for _ in range(num_vehicles):
        vehicle = None
        for attempt in range(20):  # 최대 20번 시도
            vehicle_bp = random.choice(blueprint_library.filter("model3"))

            # base_point 기준으로 무작위 위치 생성
            x_offset = random.uniform(-15, 15)
            y_offset = random.uniform(-15, 15)
            new_loc = carla.Location(
                x=base_loc.x + x_offset,
                y=base_loc.y + y_offset,
                z=base_loc.z
            )

            # base_point 자체와 너무 가까우면 제외
            if new_loc.distance(base_loc) < min_distance:
                continue

            # 기존 차량들과 너무 가까우면 제외
            if is_too_close(attempted_locations, new_loc, min_distance):
                continue

            # 차량 스폰 시도
            transform = carla.Transform(new_loc, base_point.rotation)
            vehicle = world.try_spawn_actor(vehicle_bp, transform)

            if vehicle:
                vehicle.set_autopilot(True)
                spawned_vehicles.append(vehicle)
                attempted_locations.append(new_loc)
                print(f"✅ 차량 스폰 성공 at {new_loc}")
                break
            else:
                print(f"🚫 스폰 실패 ({attempt+1}/20) at {new_loc}")

        if vehicle is None:
            print("⚠️ 해당 차량은 스폰 실패 (최대 시도 횟수 초과)")

    return spawned_vehicles

# 차량에 후방 카메라 설치
def attach_rear_camera(world, blueprint_library, vehicle):
    camera_bp = blueprint_library.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640')
    camera_bp.set_attribute('image_size_y', '480')
    camera_bp.set_attribute('fov', '110')

    # 차량 후방 카메라 위치 (차량의 뒷부분에, 뒤쪽 방향을 보도록 설정)
    camera_transform = carla.Transform(
        carla.Location(x=-5.5, z=2.0),  # 차량 뒤쪽 (x 음수) + 약간 위 (z)
        carla.Rotation(pitch=0, yaw=180, roll=0)  # 뒤쪽을 보게 yaw=180도
    )

    camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
    return camera

# 후방 카메라에 YOLOv8 모델 적용
model = YOLO(r"D:\project\yolo\0.9.15\yolov8s_finetuned2\weights\best.pt")

def process_frame(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))[:, :, :3]  # RGB only
    results = model(array)

    # # 결과 확인 (예시: bounding box 시각화)
    # result_img = results[0].plot()  # YOLOv8에서 기본 시각화 제공
    # cv2.imwrite(file_name, result_img)
    cv2.waitKey(1)  # 키 입력 기다림 없이 계속 진행

In [18]:
# 구급차 감지 여부 확인
def is_ambulance_detected(results):
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0].item())
            class_name = r.names[cls_id]
            if class_name.lower() == "ambulance":  # 구급차로 fine-tuning된 class명
                return True
    return False

# 차선 변경 로직
def change_lane(vehicle, direction="left", duration=30):
    control = carla.VehicleControl()

    if direction == "left":
        control.steer = -0.5
    elif direction == "right":
        control.steer = 0.5

    control.throttle = 0.5  # 속도 유지
    control.brake = 0.0

    for _ in range(duration):
        vehicle.apply_control(control)
        world.tick()  # 동기화된 world일 경우 사용

# 후방 카메라에 YOLOv8 모델 적용 및 구급차 감지 후 차선 변경 로직 통합
def process_frame_with_control(image, vehicle):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))[:, :, :3]
    results = model(array)

    if is_ambulance_detected(results):
        print("🚨 구급차 감지됨! 차선 변경 시도 중...")
        change_lane(vehicle, direction="right")  # 오른쪽 차선으로 회피
    else:
        print("✅ 구급차 없음, 정상 주행")

# 일반 차량 스폰 및 자율주행 활성화 + 후방 카메라에 YOLO 적용
spawned = spawn_general_vehicles(world, blueprint_library, start_transform, num_vehicles=10)
rear_cameras = []
for vehicle in spawned:
    cam = attach_rear_camera(world, blueprint_library, vehicle)
    cam.listen(lambda image: process_frame(image))
    rear_cameras.append(cam)

✅ 차량 스폰 성공 at Location(x=-35.569901, y=30.149559, z=2.027854)
✅ 차량 스폰 성공 at Location(x=-14.655054, y=26.239431, z=2.027854)
✅ 차량 스폰 성공 at Location(x=-39.681553, y=27.161531, z=2.027854)
✅ 차량 스폰 성공 at Location(x=-25.834961, y=14.860455, z=2.027854)
🚫 스폰 실패 (1/20) at Location(x=-30.803745, y=34.141598, z=2.027854)
✅ 차량 스폰 성공 at Location(x=-31.931763, y=16.883751, z=2.027854)
✅ 차량 스폰 성공 at Location(x=-21.653612, y=22.050657, z=2.027854)
✅ 차량 스폰 성공 at Location(x=-29.664181, y=36.285496, z=2.027854)
🚫 스폰 실패 (1/20) at Location(x=-28.596893, y=41.289497, z=2.027854)
🚫 스폰 실패 (2/20) at Location(x=-19.880571, y=41.751801, z=2.027854)
✅ 차량 스폰 성공 at Location(x=-39.176235, y=34.732765, z=2.027854)
✅ 차량 스폰 성공 at Location(x=-18.990654, y=15.779269, z=2.027854)
✅ 차량 스폰 성공 at Location(x=-31.101696, y=23.285976, z=2.027854)

Ultralytics 8.3.100  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

Ultralytics 8.3.100  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 30

In [19]:
# # ✅ 셀 6: 차량 주행 루프 + 조감도 카메라
# print("🚗 차량 이동 시작!")

# while True:
#     if agent.done():
#         print("✅ 목적지 도착 완료!")
#         break

#     control = agent.run_step()
#     vehicle.apply_control(control)

#     # 위에서 부드럽게 따라가는 카메라 (z 고정, yaw 고정)
#     transform = vehicle.get_transform()
#     vehicle_loc = transform.location
#     spectator.set_transform(carla.Transform(
#         carla.Location(x=vehicle_loc.x, y=vehicle_loc.y, z=50),
#         carla.Rotation(pitch=-90, yaw=0)
#     ))

#     time.sleep(0.05)

In [ ]:
# ✅ 셀 7: 차량 제거 및 정리
vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))
time.sleep(2.0)
vehicle.destroy()
print("🧹 차량 제거 완료")


Ultralytics 8.3.100  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
🧹 차량 제거 완료


0: 480x640 (no detections), 70.0ms
Speed: 4.9ms preprocess, 70.0ms inference, 18.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 87.0ms
Speed: 3.9ms preprocess, 87.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 83.3ms
Speed: 5.0ms preprocess, 83.3ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 91.5ms
Speed: 4.1ms preprocess, 91.5ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 87.3ms
Speed: 5.3ms preprocess, 87.3ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 100.3ms
Speed: 4.1ms preprocess, 100.3ms inference, 5.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 77.9ms
Speed: 3.1ms preprocess, 77.9ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 83.8ms
Speed: 3.5ms preprocess, 83.8ms

: 